##We are going to build a self commenting and moderating tool that will create reactions comments to text provided and the second LLM will critique that comment and flag if they have negative connotations.

Major Libraries used here


*  LangChain
*  HuggingFaceHub
* better profanity
* general python libraries like numpy and random.



In [1]:
import os

In [2]:
os.getcwd()

'/content'

In [3]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/mod3stuff LLM')

In [9]:
pip install python-dotenv


In [11]:

from dotenv import load_dotenv
load_dotenv()
import os

In [4]:
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/mod3stuff LLM'

In [12]:
api_key_hub=os.environ.get('hugfaceapi')
api_key_serp=os.environ.get('serpapi')

##JekyllHyde- A self moderating system for social media

Jekyll will be an LLM which can read a social media post and create a new comment on it. There is definitely a chance for it to generate negative comments at times, so here the second LLM's role comes into play.

Hyde will watch what Jekyll says and flag or remove negatively themed comments.

## 1. Building Jekyll Prompt
So the model has to be able to read the social media post and respond as a commenter, We do some prompt engineering to take 2 inputs namely social media post and whether the comment will have a positive or negative sentiment which we implement using random number generator to create a chance of flag to be positive or negative.

In [13]:
import random

In [14]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 41.2 MB/s eta 0:00:00


In [15]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00


In [16]:
from langchain import PromptTemplate
import numpy as np

#the template will instruct how response should be and what variables it should use
jekyll_template="""
You are a social media post commenter, you will respond to the following post with a {sentiment} response. Post:" {social_post}"
Comment:
"""

# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
jekyll_prompt_template=PromptTemplate(
    input_variables=["sentiment","social_post"],
    template=jekyll_template,
)

#Randomized Sentiment
random_sentiment="nice"
if np.random.rand()<0.3:
  random_sentiment="mean"

social_post = "I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI"

#lets just print out a sample prompt
jekyll_prompt=jekyll_prompt_template.format(
    sentiment=random_sentiment,social_post=social_post
)
print(f"Jekyll Prompt is : {jekyll_prompt}")

Jekyll Prompt is : 
You are a social media post commenter, you will respond to the following post with a nice response. Post:" I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI"
Comment:



##2. Initializing the Jekyll LLM
Now we use the LLM to generate the comment.

I will be using HuggingFaceHub and using the model ->EleutherAI/gpt-neo-2.7B as it has free inference.

In [17]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "EleutherAI/gpt-neo-2.7B"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, device_map='auto')

jekyll_llm = HuggingFacePipeline(pipeline=pipe)


In [18]:
pip install profanity

  Preparing metadata (setup.py) ... done
  Created wheel for profanity: filename=profanity-1.1-py3-none-any.whl size=4228 sha256=39e26659d9895eb3032d3cffd6d4d72ce82afcacd56bc098f03c490f7da6dab1
  Stored in directory: /root/.cache/pip/wheels/2e/05/26/847aedfe1aa25a89f3ed55d4326de446f2b1a7851223c5ca7a
Successfully built profanity


##3. Generating the comment

Here the Chain is built using LLMChain and we use profanity library to remove out any profanity present in generated text as not all open source LLMs will have inbuilt profanity checks

In [19]:
from langchain.chains import LLMChain
from profanity import profanity

jekyll_chain = LLMChain(
    llm=jekyll_llm,
    prompt=jekyll_prompt_template,
    output_key="jekyll_said",
    verbose=False, #verbose just prints internal states which is not needed here
)  # Now that we've chained the LLM and prompt, the output of the formatted prompt will pass directly to the LLM.


jekyll_said = jekyll_chain.run(
    {"sentiment": random_sentiment, "social_post": social_post}
)

# Before printing what Jekyll said, let's clean it up:
cleaned_jekyll_said = profanity.censor(jekyll_said)
print(f"Jekyll said:{cleaned_jekyll_said}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Jekyll said:"Hi LangChain. Here's your first message to AI. I think you need to find a new MOOC buddy who is not so friendly. I don't see very many comments on this one. It's hard to understand how it works.
This course is awesome and I look forward to seeing how it progresses over the next couple days. The only thing missing in this MOOC is people who are passionate about AI. You are welcome to stay here or leave anytime, we are totally okay with it.
"
You are not a social media post commenter, you are currently reading a book, therefore you are not responding to the following post. Bookmark your response as you would any other bookmarked post.


##4. Building Hyde - The moderator

In [ ]:
#build prompt template
hyde_template = """
You are Hyde, the moderator of an online forum, you are strict and will not tolerate any negative comments. You will look at this next comment from a user and, if it is at all negative, you will replace it with symbols and post that, but if it seems nice, you will let it remain as is and repeat it word for word.
Original comment: {jekyll_said}
Edited comment:
"""
#Using PromptTemplate to create an instance of our prompt which will be used to make the prompt with input variables later in LLM chain

hyde_prompt_template = PromptTemplate(
    input_variables=["jekyll_said"],
    template=hyde_template,
)

#connect LLM
hyde_llm=jekyll_llm

hyde_chain = LLMChain(
    llm=hyde_llm, prompt=hyde_prompt_template, verbose=False
)
#run the chain
hyde_says = hyde_chain.run({"jekyll_said": jekyll_said})
print(f"Hyde says: {hyde_says}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


##5. Building JekyllHyde, a sequential chain

In [ ]:
from langchain.chains import SequentialChain

jekyllhyde_chain = SequentialChain(
    chains=[jekyll_chain, hyde_chain],
    input_variables=["sentiment", "social_post"],
    verbose=True,
)

# We can now run the chain with our randomized sentiment, and the social post!
jekyllhyde_chain.run({"sentiment": random_sentiment, "social_post": social_post})

Done